In [2]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.preprocessing import image
import os
import matplotlib.pyplot as plt
import cv2

In [3]:
img_dims = 64
batch_size = 32


In [5]:
import tensorflow as tf

In [6]:
classifier = tf.keras.Sequential()
classifier.add(Conv2D(32, (3, 3), input_shape = (img_dims, img_dims, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))


In [7]:
input_path = 'chest_xray/'
train_datagen = ImageDataGenerator(rescale = 1./255,
shear_range = 0.2,
zoom_range = 0.2,
horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory(directory=input_path+'train',
target_size = (img_dims, img_dims),
batch_size = batch_size,
class_mode = 'binary')
test_set = test_datagen.flow_from_directory(directory=input_path+'test',
target_size = (img_dims, img_dims),
batch_size = batch_size,
class_mode = 'binary')


Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [8]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [9]:
epochs = 10

hist = classifier.fit_generator(
           training_set, steps_per_epoch=training_set.samples // batch_size, 
           epochs=epochs, validation_data=test_set, 
           validation_steps= test_set.samples)


c:\python\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
163/163 [==============================] - 359s 2s/step - loss: 0.6982 - accuracy: 0.7445 - val_loss: 0.4325 - val_accuracy: 0.7869
Epoch 2/10
163/163 [==============================] - 127s 782ms/step - loss: 0.2550 - accuracy: 0.8924
Epoch 3/10
163/163 [==============================] - 126s 773ms/step - loss: 0.2334 - accuracy: 0.9011
Epoch 4/10
163/163 [==============================] - 126s 771ms/step - loss: 0.1818 - accuracy: 0.9220
Epoch 5/10
163/163 [==============================] - 126s 771ms/step - loss: 0.1816 - accuracy: 0.9253
Epoch 6/10
163/163 [==============================] - 128s 785ms/step - loss: 0.1792 - accuracy: 0.9317
Epoch 7/10
163/163 [==============================] - 130s 795ms/step - loss: 0.1564 - accuracy: 0.9420
Epoch 8/10
163/163 [==============================] - 129s 791ms/step - loss: 0.1660 - accuracy: 0.9331
Epoch 9/10
163/163 [==============================] - 133s 812ms/step - loss: 0.1775 - accuracy: 0.9287
Epoch 10/10
163/163 [====

In [10]:
img = plt.imread('chest_xray/val/PNEUMONIA/person1954_bacteria_4886.jpeg')
img = cv2.resize(img, (img_dims, img_dims))
img = np.dstack([img, img, img])
img = img.astype('float32') / 255
result = classifier.predict(np.expand_dims(image.img_to_array(img), axis = 0))  

if result[0][0] > 0.5:
  prediction = 'Pnuemonia'
else:
  prediction = 'Normal'
  
print(prediction)

Pnuemonia


In [11]:
img = plt.imread('chest_xray/val/NORMAL/NORMAL2-IM-1427-0001.jpeg')
img = cv2.resize(img, (img_dims, img_dims))
img = np.dstack([img, img, img])
img = img.astype('float32') / 255
result = classifier.predict(np.expand_dims(image.img_to_array(img), axis = 0))  

if result[0][0] > 0.5:
  prediction = 'Pnuemonia'
else:
  prediction = 'Normal'
  
print(prediction)

Normal


In [3]:
test_data = []
test_labels = []
for cond in ['/NORMAL/', '/PNEUMONIA/']:
        for img in (listdir(input_path + 'test' + cond)):
            img = plt.imread(input_path+'test'+cond+img)
            img = cv2.resize(img, (img_dims, img_dims))
            img = np.dstack([img, img, img])
            img = img.astype('float32') / 255
            if cond=='/NORMAL/':
                label = 0
            elif cond=='/PNEUMONIA/':
                label = 1
            test_data.append(img)
            test_labels.append(label)
        
test_data = np.array(test_data)
test_labels = np.array(test_labels)
from sklearn.metrics import accuracy_score, confusion_matrix

preds = classifier.predict(test_data)

acc = accuracy_score(test_labels, np.round(preds))*100
cm = confusion_matrix(test_labels, np.round(preds))
tn, fp, fn, tp = cm.ravel()

print('CONFUSION MATRIX ------------------')
print(cm)
print('\nTEST METRICS ----------------------')
precision = tp/(tp+fp)*100
recall = tp/(tp+fn)*100
print('Accuracy: {}%'.format(acc))
print('Precision: {}%'.format(precision))
print('Recall: {}%'.format(recall))
print('F1-score: {}'.format(2*precision*recall/(precision+recall)))

print('\nTRAIN METRIC ----------------------')
print('Train acc: {}'.format(np.round((hist.history['acc'][-1])*100, 2)))


NameError: name 'listdir' is not defined